In [11]:
import os
import spacy
import pinecone
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Pinecone
from tqdm.autonotebook import tqdm


With tranformer model embeddings

In [12]:
from sentence_transformers import SentenceTransformer
model_name='all-MiniLM-L6-v2'
model = SentenceTransformer(model_name, device='cpu') 
model

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

c:\Users\muska\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\muska\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False})
  (2): Normalize()
)

In [10]:
filepath="Survey_of_MLAlgorithmsfor6g.pdf"
file_loader = PyPDFLoader(os.path.join(os.getcwd(),filepath))
documents = file_loader.load()

In [24]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=700, chunk_overlap=30)
chunks = text_splitter.split_documents(documents)
print(len(chunks))

71


In [25]:
chunks

[Document(page_content='A Survey of Machine Learning Algorithms for \n6G Wireless Networks  \n \nAnita Patil  \nDepartment of ECE, S G Balekundri Institute of Technology, Belagavi, KA,  India - 590010 .  \nanitap@sgbit.edu.in  \n \n \nSridhar Iyer  \nDepartment of ECE, KLE Dr. M.S. Sheshgiri Coll ege of Engineering and Technology , Belagavi, KA, \nIndia - 590008 .  \nsridhariyer1983@klescet.ac.in    \n \n \nRahul Jashvantbhai  Pandya  \nIndian Institute of Technology,  Dharwad, WALMI Campus, PB Road, KA,  India - 580011.  \nrpandya@iitdh.ac.in  \n \n \nAbstract:   \n        The primary focus of Artificial Intelligence/Machine Learning ( AI/ML ) integration within the wirel ess', metadata={'source': 'C:\\muskan\\IIITA\\Deep learning\\Chatbot project-Questionpro\\A Survey of Machine Learning Algorithms for 6g wirless netwok.pdf', 'page': 0}),
 Document(page_content='technology is to reduce capital expenditures, optimize network performance, and build new  revenue streams. \nReplacing tra

In [26]:
embeddings = {"id":[],"embed":[]}
for i in range(len(chunks)):
    chunk_embedding=model.encode(chunks[i].page_content).tolist()
    if len(chunk_embedding)!=0:
        embeddings['id'].append(i)
        embeddings['embed'].append(list(chunk_embedding))
print(len(embeddings['id']))

71


In [37]:
print(embeddings["id"])

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70]


In [ ]:
import pinecone
import os
from pinecone import Pinecone as pc


p = pc(    api_key="3399662c-37e6-43fa-8f62-706d2ba6050f")
index=p.Index("chatbot")

# index = Pinecone.from_documents(chunks, embeddings, index_name="chatbot-ai")
index.describe_index_stats()

: 

In [ ]:
for i in range(len(chunks)):
    # print(i)
    index.upsert(vectors=[{
        "id": str(embeddings["id"][i]),
        "values": embeddings["embed"][i]
        }],
        namespace="data_ml")
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.00071,
 'namespaces': {'data_ml': {'vector_count': 71}},
 'total_vector_count': 71}

In [ ]:
question="For what all tasks, ML algorithms are useful?"
query_embedding = model.encode(question).tolist()
# query_embedding=list(map(float, query_embedding))
xc = index.query(vector=query_embedding, top_k=3,namespace='data_ml')
xc


{'matches': [{'id': '10', 'score': 0.652693093, 'values': []},
             {'id': '14', 'score': 0.600086331, 'values': []},
             {'id': '6', 'score': 0.515233874, 'values': []}],
 'namespace': 'data_ml',
 'usage': {'read_units': 5}}

In [ ]:
for i in xc['matches']:
    if i['score']<0.3:
        print("I don't know the answer")
        break
    else:
        doc_id=i['id']
        print(chunks[int(doc_id)])
##ALL three contain relevant answers

page_content='communication.  ML is a component of AI although it endeavors to solve the problems based on \nhistorical or previous examples. Unlike AI applications, ML involves learning of hidden patterns \nwithin the data (data mining) and subsequently using the patterns to classify or pre dict an event \nrelated to the problem . Simply  put, intelligent machines depend on the knowledge to sustain their \nfunctionalities and  ML offers the same . In essence, ML algorit hms are embedded into machines and \ndata streams provided so that knowledge and information are extracted and fed into the system for \nfaster and ef ficient management of processes (Ali et. al., 2020) .' metadata={'source': 'C:\\muskan\\IIITA\\Deep learning\\Chatbot project-Questionpro\\A Survey of Machine Learning Algorithms for 6g wirless netwok.pdf', 'page': 2}
page_content='as, data security, optimization, power -efficiency , cost-efficiency, etc. Following are the ML \nalgorithms which h elp in addressing all th

In [ ]:
question="How federated learning is better than machine learning?"
query_embedding = model.encode(question).tolist()
# query_embedding=list(map(float, query_embedding))
xc = index.query(vector=query_embedding, top_k=3, namespace='data_ml')
xc

{'matches': [{'id': '7', 'score': 0.617339313, 'values': []},
             {'id': '30', 'score': 0.599694, 'values': []},
             {'id': '67', 'score': 0.533954561, 'values': []}],
 'namespace': 'data_ml',
 'usage': {'read_units': 5}}

In [ ]:
for i in xc['matches']:
    doc_id=i['id']
    print(chunks[int(doc_id)])
##Second one contains the exact answer.

page_content='algorithms work in three different versions viz., supervised learning, unsupervised learning, and \nreinforcement learning. Few ML models such as, non -parametric Baye sian methods (Gaussian \napproach), are promising, especially in handling small, incrementally growing data sets; however, \nthey have increased complexity compared to the parametric methods. Further, the Kernel Hilbert \nSpace -based solutions have shown encoura ging results in generating improved data rate, which is \n10-100 times higher in comparison to the ones shown in the 5G wireless networks, simultaneously \nbeing computationally simple and scalable with lower approximation error. Federated Learning (FL)' metadata={'source': 'C:\\muskan\\IIITA\\Deep learning\\Chatbot project-Questionpro\\A Survey of Machine Learning Algorithms for 6g wirless netwok.pdf', 'page': 1}
page_content='8. Safe Reinforcement Learning  \nSafe R L is the process of learning policies that maximize the expectation of the return